In [1]:
%reload_ext autoreload
%autoreload 2

from collections import Counter
import numpy as np
import pandas as pd
from app import Preprocessing

from matplotlib import pyplot as plt
import torch
import torch.nn as nn

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

%load_ext Cython

In [118]:
pre = Preprocessing('IMDB')

In [119]:
pre.data['train_raw'].head()

,text,POS__NEG
0,WOW this movie was so horrible Im so glad i di...,0
1,I for one really like this movie for some reas...,1
2,What an amazing film With very little dialogue...,1
3,ANDY HARDY MEETS DEBUTANTE 1940 is the ninth 9...,0
4,wonderful movie with good story great humour s...,1


In [120]:
len(list(pre.dict))

11098

In [121]:
hans = pre.build_df('train_raw','train',pre.dict)

In [122]:
pre.data['train'].describe().loc[['max'], :].sort_values(by=['max'],axis=1)

,and the story,a sad,surrounded by,The cinematography,it it,are in the,on any,home to,like me,Quite,...,of the,is,a,of,to,and,the,,,
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,50.0,68.0,70.0,80.0,85.0,98.0,176.0,277.0,279.0,281.0


In [124]:
hans = pre.build_df('test_raw','test',pre.dict)

In [310]:
x_test = torch.tensor(pre.data['test'].values).float()
y_test = torch.tensor(pre.get('test_raw')['POS__NEG'].values).long()

In [314]:
x = torch.tensor(pre.data['train'][:3000].values).float()
y = torch.tensor(pre.get('train_raw')['POS__NEG'][:3000].values).long()

In [315]:
x

tensor([[ 3.,  4.,  4.,  ...,  0.,  0.,  0.],
        [18., 18., 12.,  ...,  0.,  0.,  0.],
        [ 1.,  1.,  2.,  ...,  0.,  0.,  0.],
        ...,
        [32., 11., 16.,  ...,  0.,  0.,  0.],
        [24., 17., 18.,  ...,  0.,  0.,  0.],
        [14.,  7.,  3.,  ...,  0.,  0.,  0.]])

In [316]:
# Logistic regression model
H1 = int(2000)
H2 = int(1000)
H3 = int(500)
H4 = int(400)
H5 = int(300)
H6 = int(200)

model = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.ReLU(),
#    torch.nn.Linear(H3, H4),
#    torch.nn.ReLU(),
#    torch.nn.Linear(H4, H5),
#    torch.nn.ReLU(),
    torch.nn.Linear(H3, 2)
)

In [317]:
# Hyper-parameters 
learning_rate = 0.0001

In [318]:
# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally

criterion = nn.CrossEntropyLoss() #nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
loss_hist = []
# Train
for t in range(100):
    # Berechne die Vorhersage (foward step)
    outputs = model(x)
    
    # Berechne den Fehler (Ausgabe des Fehlers alle 50 Iterationen)
    loss = criterion(outputs, y)
    
    # Berechne die Gradienten und Aktualisiere die Gewichte (backward step)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Berechne den Fehler (Ausgabe des Fehlers alle 100 Iterationen)
    if t % 20 == 0:
        loss_hist.append(loss.item())
        print(t, loss.item())

0 0.6933587789535522
20 0.12852512300014496
40 0.0011196943232789636
60 0.00025087277754209936


In [ ]:
plt.plot(loss_hist);

In [ ]:
outputs_test = model(x_test)
#loss = criterion(outputs_test, y_test)
#loss

In [ ]:
y_test_pred_sm = outputs_test.argmax(1)

In [ ]:
# Klassifikationsgenauigkeit
classification_accuracy_test =(y_test.float() == y_test_pred_sm.float()).float().mean()
classification_accuracy_test